#What is TfIdfVectorizer?

TF (Term Frequency): The number of times a word appears in a document is its Term Frequency. A higher value means a term appears more often than others, and so, the document is a good match when the term is part of the search terms.

IDF (Inverse Document Frequency): Words that occur many times a document, but also occur many times in many others, may be irrelevant. IDF is a measure of how significant a term is in the entire corpus.

The TfidfVectorizer converts a collection of raw documents into a matrix of TF-IDF features.

In [2]:
import zipfile

zip_ref = zipfile.ZipFile("news.zip")
zip_ref.extractall()
zip_ref.close()

##Import all necessary libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

##Preprocess the data

In [4]:
#Read the data and read the first 5 head
df = pd.read_csv("news.csv")
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [5]:
#Get the shape of the data
df.shape

(6335, 4)

In [6]:
#Get the labels from the DataFrame
labels = df["label"]
labels.head()

0    FAKE
1    FAKE
2    REAL
3    FAKE
4    REAL
Name: label, dtype: object

In [7]:
#Split the dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(df["text"], labels, test_size = 0.2, random_state = 7)

In [8]:
X_train.shape, X_test.shape

((5068,), (1267,))

In [9]:
y_train.shape, y_test.shape

((5068,), (1267,))

In [10]:
X_train[:5]

6237    The head of a leading survivalist group has ma...
3722    ‹ › Arnaldo Rodgers is a trained and educated ...
5774    Patty Sanchez, 51, used to eat 13,000 calories...
336     But Benjamin Netanyahu’s reelection was regard...
3622    John Kasich was killing it with these Iowa vot...
Name: text, dtype: object

In [11]:
y_train[:5]

6237    FAKE
3722    FAKE
5774    FAKE
336     REAL
3622    REAL
Name: label, dtype: object

##Initialize TfidfVectorizer

Let’s initialize a TfidfVectorizer with stop words from the English language and a maximum document frequency of 0.7 (terms with a higher document frequency will be discarded). Stop words are the most common words in a language that are to be filtered out before processing the natural language data. And a TfidfVectorizer turns a collection of raw documents into a matrix of TF-IDF features.

In [12]:
tfidf_vectorizer = TfidfVectorizer(stop_words = 'english', max_df = 0.7)

#Fit and transform train and test set
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

In [13]:
#Initialize the PassiveAgressiveClassifier
pac = PassiveAggressiveClassifier(max_iter = 50)
pac.fit(tfidf_train, y_train)

PassiveAggressiveClassifier(max_iter=50)

In [14]:
#Predict on the test set and calculate accuracy
y_pred=pac.predict(tfidf_test)
score=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 93.05%


We got an accuracy of 92.82% with this model. Finally, let’s print out a confusion matrix to gain insight into the number of false and true negatives and positives.

##Build confusion matrix

In [15]:
confusion_matrix(y_true = y_test,
                 y_pred = y_pred,
                 labels = ["FAKE", "REAL"])

array([[594,  44],
       [ 44, 585]])

We have 589 true positives, 587 true negatives, 42 false positives, and 49 false negatives